In [15]:

import pandas as pd
import os
from IPython.display import display
from pandasql import sqldf
import numpy as np
from scipy import stats
import plotly.express as px
dataset_path = 'dataset'
pysqldf = lambda q: sqldf(q, globals())


silver_df = pd.read_parquet(os.path.join(dataset_path, 'silver_df.parquet'))



In [16]:
query_df = silver_df.loc[silver_df.Year > 1960]
query_df['Medal'] = query_df['Medal'].astype(pd.CategoricalDtype(categories=['Gold', 'Silver', 'Bronze', 'NoMedal'], ordered=True))
query_df['Medal'] = query_df['Medal'].fillna(value='NoMedal')


C:\Users\andre\AppData\Local\Temp/ipykernel_18732/136624745.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andre\AppData\Local\Temp/ipykernel_18732/136624745.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:


# NOC, year, and different medals
noc_height_df = pd.pivot_table(query_df,
                               index=['Height', 'Sex'],
                               columns='Medal',
                               aggfunc='count',
                               values='Event',
                               fill_value=0,
                               margins=True,
                               margins_name='TotalPartecipants',
                               observed=True)
noc_height_df = noc_height_df.drop(('TotalPartecipants',''))
noc_height_df.columns.name = ''
noc_height_df.reset_index(inplace=True)
noc_height_df.Height = noc_height_df.Height.astype(int)
display(noc_height_df.describe())

noc_height_df.to_parquet(os.path.join('dataset','noc_height.parquet'))

threshold = noc_height_df.TotalPartecipants.quantile(0.15)
noc_height_df = noc_height_df[noc_height_df.TotalPartecipants>threshold]
noc_height_df['TotalMedals'] = noc_height_df.Gold + noc_height_df.Silver + noc_height_df.Bronze
noc_height_df['MedalPerPartecipant'] = np.divide(noc_height_df.TotalMedals, noc_height_df.TotalPartecipants)
print(f"Filter out heights that have less than {threshold:.1f} datapoints")


,Height,Gold,Silver,Bronze,NoMedal,TotalPartecipants
count,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000
mean,173.538462,52.739645,52.224852,54.171598,963.224852,1122.360947
std,25.634333,75.039852,74.570654,76.897660,1469.951452,1684.239232
min,127.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,152.000000,1.000000,1.000000,1.000000,12.000000,16.000000
50%,173.000000,15.000000,15.000000,15.000000,180.000000,231.000000
75%,194.000000,89.000000,85.000000,83.000000,1395.000000,1783.000000
max,226.000000,357.000000,344.000000,405.000000,8358.000000,9455.000000


Filter out heights that have less than 5.2 datapoints


In [11]:
fig = px.scatter(x=noc_height_df.Height, y=noc_height_df['MedalPerPartecipant'], color=noc_height_df.Sex)
# fig.update_layout(title='Height probability density based on sex and year',
#                    yaxis_title='PDF (-)',
#                    xaxis_title='Height (cm)',
#                   legend={'title': 'Sex:'})
fig.show()

In [12]:
male_df = noc_height_df.loc[noc_height_df.Sex == "M"]
female_df = noc_height_df.loc[noc_height_df.Sex == "F"]


In [18]:
print('Male:')
male_df.corr(method='pearson')

Male:


,Height,Gold,Silver,Bronze,NoMedal,TotalPartecipants,TotalMedals,MedalPerPartecipant
,,,,,,,,
Height,1.000000,-0.000557,-0.016164,-0.029777,-0.127310,-0.113931,-0.015718,0.765041
Gold,-0.000557,1.000000,0.981478,0.977061,0.912160,0.928297,0.992937,-0.269744
Silver,-0.016164,0.981478,1.000000,0.979977,0.938907,0.951932,0.993872,-0.299603
Bronze,-0.029777,0.977061,0.979977,1.000000,0.956229,0.967002,0.992624,-0.315637
NoMedal,-0.127310,0.912160,0.938907,0.956229,1.000000,0.999094,0.942384,-0.404226
TotalPartecipants,-0.113931,0.928297,0.951932,0.967002,0.999094,1.000000,0.955769,-0.393204
TotalMedals,-0.015718,0.992937,0.993872,0.992624,0.942384,0.955769,1.000000,-0.297183
MedalPerPartecipant,0.765041,-0.269744,-0.299603,-0.315637,-0.404226,-0.393204,-0.297183,1.000000


In [14]:
print('Female:')
female_df.corr(method='pearson')

print(stats.pearsonr(female_df.Height, female_df.MedalPerPartecipant))

Female:
(0.7881405282362357, 9.22524710646056e-16)


In [19]:
print(stats.pearsonr(male_df.Height, male_df.MedalPerPartecipant))



(0.7650405986895926, 2.1034212870634873e-15)
